In [ ]:
import pandas as pd 
df = pd.read_csv('ogsample.csv')

from nltk.tokenize import word_tokenize

texts = df['Content']

tokenized_texts = [word_tokenize(text) for text in texts]

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Preprocess the text
df['tokens'] = df['Content'].apply(simple_preprocess)

# Train Word2Vec model
model = Word2Vec(sentences=df['tokens'], vector_size=300, window=5 , min_count=15, workers=4, sg=0, epochs=10)

# Save the model for later use
model.save("word2vec.model")

In [ ]:
print(model.wv.key_to_index)

In [ ]:
# Evaluate embeddings
print(model.wv.most_similar('woman'))

In [ ]:
# Evaluate embeddings
print(model.wv.most_similar('man'))

In [ ]:
# Evaluate embeddings
print(model.wv.most_similar('government'))

In [ ]:
# Calculate the cosine similarity between "man" and "woman"
similarity = model.wv.similarity('man', 'woman')

print(f"Cosine similarity between 'man' and 'woman': {similarity}")

In [ ]:
# Calculate the cosine similarity between "man" and "woman"
similarity = model.wv.similarity('male', 'female')

print(f"Cosine similarity between 'male' and 'female': {similarity}")

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
model = Word2Vec(sentences=df['tokens'], vector_size=300, window=5 , min_count=15, workers=4, sg=0, epochs=10)

# Save the model for later use
model.save("word2vec.model")

In [ ]:
# Define target and attribute word sets
career_words = ['parliament', 'conservative', 'member', 'election', 'labour', 'vote', 'party', 'parliamentary', 'mp', 'government', 
                 'work', 'become', 'serve', 'state', 'elect', 'seat', 'support', 'campaign', 'career', 'public', 'minister', 'first', 
                 'candidate', 'record', 'leader', 'secretary', 'whip', 'political', 'brexit', 'secretary', 'lord', 'common'
                 'leadership', 'local', 'take', 'announce', 'shadow', 'majority', 'constituency', 'contribution', 
                 'office', 'liberal', 'politician', 'policy', 'resign', 'hold', 'committee', 'service', 
                 'position', 'lose', 'select', 'lead', 'voting', 'contest', 'defeat', 'chair', 'deputy',
                 'council', 'represent', 'gain', 'number', 'sit','promote', 'trade', 'replace', 'speak', 'result', 'reelect', 'post', 'oppose', 
                 'rule', 'publish', 'debate', 'raise', 'referendum', 'mlp', 'mps', 'memo']
personal_life_words = ['bear', 'personal', 'private', 'family', 'marry', 'university', 'young', 
                       'degree', 'marriage', 'name', 'death' , 'son' , 'daughter', 'college', 'friend',  
                       'die', 'couple', 'mother', 'parent', 'husband', 'girl', 'cancer', 'partner', 
                       'wife', 'father', 'doctor', 'divorce', 'race', 'actress',  'author', 'brother', 
                       'blog', 'married', 'lawyer', 'bachelor', 'character', 'birthday', 'ba', 'phd', 
                       'grandfather', 'actor', 'grandchild', 'pretty', 'ugly', 'grandparent', 'sibling', 
                       'singer', 'widow', 'childhood', 'hobby', 'surname', 'grandson', 'footballer', 'doctorate', 
                       'nickname', 'autobiography', 'twin', 'tall', 'darling', 'paternal', 'aunt', 'miscarriage', 
                       'blogger', 'granddaughter',  'lover', 'cousin', 'sisterinlaw', 'stepson']

In [ ]:
import numpy as np
   
# Load the trained Word2Vec model
model = Word2Vec.load("word2vec.model")
   
# Get the word vectors
word_vectors = model.wv

# Calculate WEAT scores
weat_scores = []
for tokens in df['tokens']:
    # Calculate cosine similarity
    cos_sim_career = np.mean([np.dot(word_vectors[word], word_vectors[token]) / (np.linalg.norm(word_vectors[word]) * np.linalg.norm(word_vectors[token])) for word in career_words for token in tokens if word in word_vectors and token in word_vectors])
    cos_sim_personal_life = np.mean([np.dot(word_vectors[word], word_vectors[token]) / (np.linalg.norm(word_vectors[word]) * np.linalg.norm(word_vectors[token])) for word in personal_life_words for token in tokens if word in word_vectors and token in word_vectors])

    # Calculate WEAT score
    weat_score = cos_sim_career - cos_sim_personal_life
    weat_scores.append(weat_score)

# Add WEAT scores to the DataFrame
df['WEAT_score'] = weat_scores

In [ ]:
df

In [ ]:
df.to_csv('ogsample2.csv')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def count_references(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    reference_list = soup.find('ol', {'class': 'references'})
    if reference_list is None:
        return 0
    else:
        return len(reference_list.find_all('li'))

# Load data from CSV
df = pd.read_csv('ogsample2.csv')  # replace 'filename.csv' with your CSV file name

# Create a new column 'reference_count' by applying the function to the 'name' column
df['reference_count'] = df['Name'].apply(lambda x: count_references('https://en.wikipedia.org/wiki/' + x))

# # Save the DataFrame to a new CSV file
# df.to_csv('output.csv', index=False)


In [ ]:
import wikipediaapi
from nltk.sentiment import SentimentIntensityAnalyzer


In [ ]:
import nltk
nltk.download('vader_lexicon')


In [ ]:
import pandas as pd
import requests

# Download required NLTK resources
nltk.download('punkt')

# # Read the CSV file containing names
# names_df = pd.read_csv(r"C:\Users\gargi\Desktop\Jupyter Notebook\Dissertation1.csv")

# Create empty lists to store the counts
positive_counts = []
negative_counts = []
neutral_counts = []
total_counts = []

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()


#Perform sentiment analysis and count sentences for each name
for name in df['Name']:
    # Fetch Wikipedia page content
    url = f"https://en.wikipedia.org/wiki/{name}"
    response = requests.get(url)
    if response.status_code == 200:
        # Extract the page content
        page_content = response.text

        # Tokenize the page content into sentences
        sentences = nltk.sent_tokenize(page_content)

        # Initialize counters for positive, negative, and neutral sentences
        pos_count = 0
        neg_count = 0
        neu_count = 0
        total_count = len(sentences)

        # Perform sentiment analysis and count sentences
        for sentence in sentences:
            sentiment_scores = sia.polarity_scores(sentence)
            if sentiment_scores['compound'] > 0:
                pos_count += 1
            elif sentiment_scores['compound'] < 0:
                neg_count += 1
            else:
                neu_count += 1
                
         # Append the counts to the respective lists
        positive_counts.append(pos_count)
        negative_counts.append(neg_count)
        neutral_counts.append(neu_count)
        total_counts.append(total_count)
    else:
        print(f"Error fetching Wikipedia page for {name}")
        
# Add the counts as new rows in the DataFrame
df['Positive Sentences'] = positive_counts
df['Negative Sentences'] = negative_counts
df['Neutral Sentences'] = neutral_counts
df['Total Sentences'] = total_counts

In [ ]:
# Filter the DataFrame to include only female politicians
female_politicians = df.loc[df['Gender'] == 'Female']
male_politicians = df.loc[df['Gender'] == 'Male']
# Calculate the mean value of positive sentences for female politicians
mean_positive_female = female_politicians['Positive Sentences'].mean()
mean_positive_male = male_politicians['Positive Sentences'].mean()
mean_negative_female = female_politicians['Negative Sentences'].mean()
mean_negative_male = male_politicians['Negative Sentences'].mean()
mean_total_female = female_politicians['Total Sentences'].mean()
mean_total_male = male_politicians['Total Sentences'].mean()

# Print the mean value
print(f"Mean positive sentences for female politicians: {mean_positive_female}")
print(f"Mean positive sentences for male politicians: {mean_positive_male}")
print(f"Mean negative sentences for female politicians: {mean_negative_female}")
print(f"Mean negative sentences for male politicians: {mean_negative_male}")
print(f"Mean total sentences for female politicians: {mean_total_female}")
print(f"Mean total sentences for male politicians: {mean_total_male}")

In [ ]:
import numpy as np
# Create a new column and assign 1 if gender is male, else assign 0
df['GenderCode'] = np.where(df['Gender'] == 'Male', 1, 0)

# Print the DataFrame
print(df)

In [ ]:
df.to_csv('ogsample3.csv')